In [22]:
 # Binary Classification with Sonar Dataset: Baseline
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json


from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline
from keras.utils import np_utils
import os

In [23]:
seed = 1992
numpy.random.seed(seed)


In [24]:
filename='ML-Long_S50M15_07ToNow-Train.csv'
filepath=os.path.abspath(f'D:\DataSets\/{filename}')
print(filepath)

D:\DataSets\ML-Long_S50M15_07ToNow-Train.csv


In [25]:
dataframe = pandas.read_csv(filepath,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
print(dataframe.info())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70693 entries, 2007-01-03 09:45:00 to 2020-06-10 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    70693 non-null  float64
 1   high                    70693 non-null  float64
 2   low                     70693 non-null  float64
 3   close                   70693 non-null  float64
 4   indy_ma-550             70693 non-null  float64
 5   indy_ma-1100            70693 non-null  float64
 6   indy_hh-550             70693 non-null  float64
 7   indy_ll-550             70693 non-null  float64
 8   indy_mid-550            70693 non-null  float64
 9   indy_hh2-1100           70693 non-null  float64
 10  indy_ll2-1100           70693 non-null  float64
 11  indy_mid2-1100          70693 non-null  float64
 12  indy_macd110-440        70693 non-null  float64
 13  indy_signal110-440-110  70693 non-null  float64
 14  ind

In [26]:
df=dataframe['01-2019':'12-2020']
df.drop(['open','high','low','close'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7838 entries, 2019-01-02 09:45:00 to 2020-06-10 16:45:00
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             7838 non-null   float64
 1   indy_ma-1100            7838 non-null   float64
 2   indy_hh-550             7838 non-null   float64
 3   indy_ll-550             7838 non-null   float64
 4   indy_mid-550            7838 non-null   float64
 5   indy_hh2-1100           7838 non-null   float64
 6   indy_ll2-1100           7838 non-null   float64
 7   indy_mid2-1100          7838 non-null   float64
 8   indy_macd110-440        7838 non-null   float64
 9   indy_signal110-440-110  7838 non-null   float64
 10  indy_hist_macd110-440   7838 non-null   float64
 11  indy_rsi25-ma20         7838 non-null   float64
 12  indy_6ATRTrail_DC-110   7838 non-null   float64
 13  cate_3trend-550_ma110   7838 non-null   int64  
 14  cate

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
labelName=df.columns[-1]
labelName

'LongSignal'

In [28]:

X_df = df.drop(columns=labelName).astype(float)
Y_df = df[labelName]

In [29]:
X_df.head(3)

,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,indy_ll2-1100,indy_mid2-1100,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,
2019-01-02 09:45:00,1076.48,1085.09,1123.2,1022.6,1072.9,1125.0,1022.6,1073.8,-29.84,-24.72,-5.12,45.39,1049.31,1.0,0.0,3.0,2.0
2019-01-02 10:00:00,1076.41,1085.02,1123.2,1022.6,1072.9,1124.6,1022.6,1073.6,-29.91,-24.80,-5.11,45.98,1049.19,1.0,0.0,3.0,2.0
2019-01-02 10:15:00,1076.35,1084.94,1123.2,1022.6,1072.9,1124.0,1022.6,1073.3,-29.97,-24.88,-5.09,46.57,1049.23,1.0,0.0,3.0,2.0


In [30]:
Y_df.head(3)

datetime
2019-01-02 09:45:00    0
2019-01-02 10:00:00    0
2019-01-02 10:15:00    0
Name: LongSignal, dtype: int64

In [31]:
# split into input (X) and output (Y) variables
X = X_df.to_numpy()
y =Y_df.to_numpy()
print(X.shape)
print(y.shape)

(7838, 17)
(7838,)


# How to Save and Reuse Data Preparation Objects in Scikit-Learn

In [32]:
from pickle import load

from sklearn.metrics import accuracy_score

import collections

In [34]:
model_path=os.path.abspath(r'D:\JupyterCode\ML-S50F\Lab-S5-F-DNN\final_model')

model_dumpfile=f'{model_path}\B1_long_MLP_17x17x0_epo105_btch32_tn2007-2018_ts2019-2020_030720_1553_Dump-Model.pkl'
scaler_dumpfile=f'{model_path}\B1_long_MLP_17x17x0_epo105_btch32_tn2007-2018_ts2019-2020_030720_1553_Dump-Scaler.pkl'

print(f'{model_dumpfile}',os.path.exists(model_dumpfile))
print(f'{scaler_dumpfile}',os.path.exists(scaler_dumpfile))

D:\JupyterCode\ML-S50F\Lab-S5-F-DNN\final_model\B1_long_MLP_17x17x0_epo105_btch32_tn2007-2018_ts2019-2020_030720_1553_Dump-Model.pkl True
D:\JupyterCode\ML-S50F\Lab-S5-F-DNN\final_model\B1_long_MLP_17x17x0_epo105_btch32_tn2007-2018_ts2019-2020_030720_1553_Dump-Scaler.pkl True


In [35]:
model = load(open(model_dumpfile, 'rb'))
# load the scaler
scaler = load(open(scaler_dumpfile, 'rb'))
# transform the test dataset


In [36]:
model

In [37]:
scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [38]:
# transform the test dataset
X_test_scaled = scaler.transform(X)
# make predictions on the test set
yPredicted = model.predict_classes(X_test_scaled)
# evaluate accuracy
acc = accuracy_score(y, yPredicted)
print('Test Accuracy:',round(acc*100,2))

Test Accuracy: 81.65
